In [1]:
import cv2
import os
import imutils
import numpy as np

import json
import random
from time import sleep

In [2]:
from IPython.display import clear_output
from google.colab.patches import cv2_imshow 
import matplotlib.pyplot as plt

In [3]:
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [4]:
import pandas as pd

In [5]:
%matplotlib inline

In [6]:
base_colab = 'drive/MyDrive/'

In [7]:
DATA_CONFIG = [
    {
        "ubiris_url" : 'drive/MyDrive/ubirisv2/CLASSES_400_300_Part1/',
        "dataset_url": "drive/MyDrive/datasets/cut_eyes/",
        "total_people": 260,
        "allowed_img_ids": [13, 14, 15],
        "allowed_session_ids": [1],
        "excluded_pepople": [53,54,55,56]
    },
    {
        "ubiris_url" : 'drive/MyDrive/ubirisv2/CLASSES_400_300_Part2/',
        "dataset_url": "drive/MyDrive/datasets/cut_eyes/",
        "allowed_img_ids": [13, 14, 15],
        "allowed_session_ids": [1],
        "excluded_pepople": [53,54,55,56]
    }
]

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#@title
def canny_edge_detect(image, min = 60, max=120):
  #Detecção de bordas por Canny
  _image = apply_blur_on_image(image)
  canny=cv2.Canny(_image, min, max)
  #canny=cv2.Canny(gray, 170, 200)
  #plt.imshow(canny,cmap=plt.cm.gray), plt.xticks([]), plt.yticks([])
  return canny

In [10]:
#@title
def apply_contourns(image, canny):
  edges, _ = cv2.findContours(canny.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
  draw = cv2.drawContours(image.copy(), edges, -1, (255,255,255), 2)
  #plt.imshow(draw,cmap=plt.cm.gray), plt.xticks([]), plt.yticks([])
  return draw

In [11]:
#@title
def create_euclidean_array(circles,x_center,y_center):
  distances = []
  #print("Image center: {}".format((x_center,y_center)))
  for circle in circles:
    #x,y = circle[0],circle[1]
    point1 = np.array((circle[0],circle[1]))
    point2 = np.array((x_center,y_center))
    
    distances.append(np.linalg.norm(point1-point2))
  return distances

In [12]:
#@title
def select_circle(circles,x_center,y_center,neighbors=3):
  _distances = []
  distances = []
  #print("Image center: {}".format((x_center,y_center)))
  _distances = create_euclidean_array(circles,x_center,y_center)
  
  indices = sorted(
    range(len(_distances)),
    key=lambda index: _distances[index]
  )
  #indices.sort(reverse=True)
  #print("best distances static center: {} {} {}".format(_distances[indices[0]], _distances[indices[1]],_distances[indices[2]] ))
  selected_circles = []
  x, y = 0 , 0
  max_circles = neighbors if len(circles) > neighbors else len(circles)
  for i in range(max_circles):
    selected_circles.append(circles[indices[i]])
    x += circles[i][0]
    y += circles[i][1]
  factor = 1 #random.uniform(0, 0.5)
  x_adjusted = int((x/max_circles - x_center) * factor + x_center)
  y_adjusted = int((y/max_circles - y_center)  + y_center)
  #print("Image center adjusted: {}".format((x,y)))
  

  distances = create_euclidean_array(circles, x_adjusted, y_adjusted)

  #print("fixed {} - adjusted {}".format(min(_distances),min(distances)))
  if (min(_distances) > min(distances)):
    mindist = min(distances)
    minpos = distances.index(mindist)
  else:
    mindist = min(_distances)
    minpos = _distances.index(mindist)
  mindist = min(distances)
  minpos = distances.index(mindist)
  return (circles[minpos], mindist)

In [107]:
def find_best_circle(draw, print_output=False, output_image= None, show_circles=False):
  _draw = draw.copy()
  #houg_settings=[[70,80],[80,90]]
  houg_settings=[[70,80],[80,90],[90,100]]
  #houg_settings=[[70,80],[80,90],[90,100], [100,110]]
  #houg_settings=[[90,100]]
  (x, y, _) = draw.shape

  center_mask = np.zeros((x,y),np.uint8)
  cv2.circle(center_mask, (int(y/2), int(x/2)), int(y/2), (255, 255, 225), -1)
  _draw = apply_mask(center_mask,_draw)

  gray = cv2.cvtColor(_draw, cv2.COLOR_BGR2GRAY)


  gray_img_blur = cv2.medianBlur(gray, 5)
  
  '''

  kernel = np.ones((5,5),np.uint8)
  
  #Morphological Transformations
  
  gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_OPEN, kernel)
  print_image(gray_img_blur)
  gray_img_blur = cv2.erode(gray_img_blur,kernel,iterations  = 1)
  print_image(gray_img_blur)
  gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_OPEN, kernel)
  print_image(gray_img_blur)
  gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_GRADIENT, kernel)
  '''
  #gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_GRADIENT, kernel)
  #gray_img_blur = cv2.erode(gray_img_blur,kernel,iterations  = 1)
  #gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_TOPHAT, kernel)
  #gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_CLOSE, kernel)
  #gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_OPEN, kernel)

  #cimg = cv2.cvtColor(gray_img_blur, cv2.COLOR_GRAY2BGR)
  best_circle = None
  best_dist = None
  best_ray = None
  
  print_image(gray_img_blur)
  best_settings = None
  for settings in houg_settings:
    print("current houg settings: {}".format(settings))
    circles = cv2.HoughCircles(gray_img_blur, cv2.HOUGH_GRADIENT, 1, gray.shape[0]/64, param1=910, param2=15, minRadius=settings[0], maxRadius=settings[1])
    
    if show_circles and circles is not None and output_image is not None:
      circles = np.uint16(np.around(circles))
      img_circles = output_image.copy()
      for i in circles[0, :]:
          # Draw outer circle
          cv2.circle(img_circles, (i[0], i[1]), i[2], (0, 255, 0), 2)
          # Draw inner circle
          cv2.circle(img_circles, (i[0], i[1]), 2, (0, 0, 255), 3)
      print_image(img_circles)
      del(img_circles)
    
    circle, dist = None, None
    #if circles is not None and len(circles[0]) > 1 :
    if circles is not None:
      print("have circles")
      circle, dist = select_circle(circles[0], int(y/2) ,int(x/2))
      if circle is not None:
        best_circle = circle
        best_dist = dist
        best_ray = circle[2]
      if best_circle is None:
        best_circle = circle
        best_dist = dist
        best_ray = circle[2]
    


    if circle is not None:
      if dist < best_dist: #and best_ray < circle[2]:
      #if dist < best_dist and best_ray < circle[2]:
        best_settings = settings
        best_dist = dist
        best_circle = circle
        best_ray = circle[2]

  if best_circle is None:
    print("Not circles found")
    return None

  best_circle = np.uint16(np.around(best_circle))
  print("houg settings: {} with ray: {}".format(best_settings, best_circle[2]))
  print("best cicle found: ({},{})".format(best_circle[0], best_circle[1]))

  xc, yc, rc = best_circle
  margin = 4
  print("{},{} ray {}".format(xc,yc,rc))
  x1 = int(xc) - int(rc) - int(margin) if int(xc) - int(rc) - int(margin) > 0 else 0
  y1 = int(yc) - int(rc) - int(margin) if int(yc) - int(rc) - int(margin) > 0  else 0

  x2 = xc+rc+ margin if xc+rc+ margin < 400 else 400
  y2 = yc + rc+ margin if yc + rc+ margin < 300 else 300

  print("({},{}) ({},{})".format(x1,y1,x2,y2))

  mask_rect = np.zeros((x,y),dtype=np.uint8)
  cv2.rectangle(mask_rect, (x1,y1), (x2,y2), (255,255,255), -1)
  cropped = apply_mask(mask_rect,output_image)
  cropped = output_image[y1:y2, x1:x2 ]
  #cv2.imwrite(argv[2],mask)
  #out = img*mask
  #white = 255-mask
  #cv2.imwrite(argv[2],out+white)
  #print_image(mask_rect)

  mask = np.zeros((x,y),np.uint8)
  cv2.circle(mask, (best_circle[0], best_circle[1]), best_circle[2], (255, 255, 225), -1)
  if output_image is not None:
    border_iris = output_image.copy()
    cv2.circle(border_iris, (best_circle[0], best_circle[1]), best_circle[2], (255, 255, 225), 2)
    cv2.circle(border_iris, (best_circle[0], best_circle[1]), 2, (0, 0, 255), 3)
    #plt.imshow(border_iris,cmap=plt.cm.gray), plt.xticks([]), plt.yticks([])
    #plt.show()
    print_image(border_iris)
  return cropped

In [14]:
#@title
def apply_mask(mask,image):
  masked = cv2.bitwise_and(image, image, mask=mask)
  return masked

In [15]:
#@title
def detect_best_circle_in(image,canny_config=(60,80),show_circles=False):
  #_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  temp = image.copy()
  _image = image.copy()
  #_canny = canny_edge_detect(image, 60, 120)
  _canny = canny_edge_detect(image, canny_config[0],canny_config[1])
  print_image(_canny)
  _image = apply_contourns(_image, _canny)
  mask = find_best_circle(_image, output_image = image.copy(),print_output=True,show_circles=show_circles)
  print("mask on main func")
  #print_image(mask)
  if mask is None:
    return image
  #print_image(image.copy())
  #iris_rounded, _, _, _, _ = apply_mask(mask,image.copy())
  
  #iris_rounded = iris_rounded[yc - rc - margin:yc + rc + margin, xc - rc - margin:xc + rc + margin]
  #iris_rounded = image.copy()*mask
  #print_image(mask)
  return mask
  

In [16]:
#@title
def save_image(directory,filename,image):
  print(directory + filename)
  if not os.path.exists(directory):
    os.makedirs(directory)
  cv2.imwrite(str(directory + filename), image)

In [17]:
#@title
def apply_blur_on_image(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  gray_img_blur = cv2.medianBlur(gray, 5)

  kernel = np.ones((5,5),np.uint8)
  
  #Morphological Transformations

  gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_OPEN, kernel)
  gray_img_blur = cv2.erode(gray_img_blur,kernel,iterations  = 1)
  gray_img_blur = cv2.morphologyEx(gray_img_blur, cv2.MORPH_OPEN, kernel)
  return gray_img_blur


In [18]:
#@title
def print_image(image):
  plt.imshow(image,cmap=plt.cm.gray), plt.xticks([]), plt.yticks([])
  plt.show()

In [19]:
#@title
def extract_info(file_name):
  file_name = file_name.split('.')[0]
  [user, session, image] = file_name.split('_')
  user_id = user.replace("C",'')
  session_id = session.replace("S",'')
  image_id = image.replace("I",'')
  return [user_id, session_id, image_id]

In [20]:
#@title
def get_all_ids():
  ids = []
  for folder in os.listdir("drive/MyDrive/datasets/base_eyes/"):
    user_id = int(folder.replace("C",""))
    if user_id in [53,54]:
      continue
    if user_id not in ids:
      #print(user_id)
      ids.append(user_id)
  
  return sorted(ids)

In [21]:
#@title
def approve_derectory():
  ids = []
  for folder in os.listdir("drive/MyDrive/datasets/cut_eyes/"):
    user_id = int(folder.replace("C",""))
    files = os.listdir("drive/MyDrive/datasets/cut_eyes/" + folder)
    if len(files) >= 3:
      #print(user_id)
      ids.append(user_id)
  
  return sorted(ids)  

In [22]:
#@title
DATA_CONFIG = [
    {
        "ubiris_url" : 'drive/MyDrive/ubirisv2/CLASSES_400_300_Part1/',
        "dataset_url": "drive/MyDrive/datasets/cut_eyes/",
        "total_people": 260,
        "allowed_img_ids": [13, 14, 15],
        "allowed_session_ids": [1],
        "excluded_pepople": [53,54,55,56]
    },
    {
        "ubiris_url" : 'drive/MyDrive/ubirisv2/CLASSES_400_300_Part2/',
        "dataset_url": "drive/MyDrive/datasets/cut_eyes/",
        "allowed_img_ids": [13, 14, 15],
        "allowed_session_ids": [1],
        "excluded_pepople": [53,54,55,56]
    }
]

In [23]:
#@title
def create_dataset():
  starts_in = 1
  rejected_eyes = 0
  removed_ids = [53,54]
  approved_ids = approve_derectory()
  
  total_eyes = len(all_ids) - len(removed_ids)
  
  base_eyes = "drive/MyDrive/datasets/base_eyes/"
  cut_eyes = "drive/MyDrive/datasets/cut_eyes/"
  for customer in all_ids:
    if customer < starts_in:
      continue
    if customer in removed_ids:
      continue
    if customer in approved_ids:
      continue
    #path = DATA_CONFIG[0]["dataset_url"] + "C{}/".format(customer) if customer <= 260 else DATA_CONFIG[1]["dataset_url"] + "C{}/".format(customer)


    #files = ['C{}_S1_I13.tiff'.format(customer), 'C{}_S2_I13.tiff'.format(customer)]
    files = os.listdir(base_eyes + "C{}".format(customer))
    
    #path = DATA_CONFIG[0]["dataset_url"] + "C{}/".format(customer)

    for picture in files:
      if os.path.exists(cut_eyes + "C{}".format(customer)):
        existent_files = os.listdir(cut_eyes + "C{}".format(customer))
        if picture in existent_files:
          continue

      print("Customer {}".format(customer))
      image_path = base_eyes + "C{}/".format(customer) + picture
      #image_path = image_path.replace("\\","/")
      print(image_path)
      original=cv2.imread(image_path)
      original=cv2.cvtColor(original, cv2.COLOR_RGB2BGR)

      iris= detect_best_circle_in(original, canny_config=(3,14))
      
      #plt.imshow(iris,cmap=plt.cm.gray), plt.xticks([]), plt.yticks([])
      #plt.show()
      print("Folder: C{} file: {} customer {}".format(customer,picture,customer))
      choice = str(input("salvar ? y/n"))
      save = True if choice == "y" else False

      if save:
        save_image(cut_eyes + "C{}/".format(customer), picture,cv2.cvtColor(iris, cv2.COLOR_BGR2RGB))
      else:
        rejected_eyes += 1
      clear_output()

      
  print("Completed {:.2f}% restant: {}".format(100*(total_eyes - rejected_eyes)/total_eyes, rejected_eyes ))


In [24]:
#@title
def count_all_files():
  count = 0
  for folder in os.listdir("drive/MyDrive/datasets/cut_eyes/"):
    count += len(os.listdir("drive/MyDrive/datasets/cut_eyes/" + folder))
  return count

In [59]:
def count_restant_files():
  count = 0
  for folder in os.listdir("drive/MyDrive/datasets/cropped_eyes/"):
    count += len(os.listdir("drive/MyDrive/datasets/cropped_eyes/" + folder))
  return count

In [81]:
def verify_large_files():
  issues = []
  cropped_eyes = "drive/MyDrive/datasets/cropped_eyes/"
  for folder in os.listdir(cropped_eyes):
    for pic in os.listdir(cropped_eyes + folder):
      image_path = cropped_eyes + folder + "/" + pic
      original=cv2.imread(image_path)
      if original.shape[0] == 300:
        issues.append({"pic": pic, "shape": original.shape})
  return issues

In [111]:
verify_large_files()

[]

In [103]:
def crop_dataset():
  cropped_eyes = "drive/MyDrive/datasets/cropped_eyes/"
  cut_eyes = "drive/MyDrive/datasets/cut_eyes/"
  rejected_eyes = 0
  approved_eyes = count_restant_files()
  total_eyes = count_all_files()
  folders = os.listdir(cut_eyes)
  folders.sort()
  for folder in folders:
    files = os.listdir(cut_eyes + folder)
    for picture in files:
      if os.path.exists(cropped_eyes + folder):
        existent_files = os.listdir(cropped_eyes + folder)
        if picture in existent_files:
          continue
      image_path = cut_eyes + folder + "/" + picture
      original=cv2.imread(image_path)
      original=cv2.cvtColor(original, cv2.COLOR_RGB2BGR)
      iris= detect_best_circle_in(original, canny_config=(7,15), show_circles=True)
      print(iris.shape)
      #if iris.shape[0] == 300:
        #rejected_eyes += 1
        #clear_output()
        #continue
      print_image(iris)
      print("{:.2f}".format(100*(approved_eyes/total_eyes)))
      print("folder {} file {}".format(folder,picture))
      choice = str(input("salvar ? y/n"))
      save = True if choice == "y" else False

      if save:
        #print(cropped_eyes + folder + "/" + picture)
        approved_eyes += 1
        save_image(cropped_eyes + folder + "/", picture,cv2.cvtColor(iris, cv2.COLOR_BGR2RGB))
      else:
        rejected_eyes += 1
      clear_output()
  print("Completed {:.2f}% restant: {}".format(100*(total_eyes - rejected_eyes)/total_eyes, rejected_eyes ))


In [115]:
crop_dataset()

Completed 100.00% restant: 0


In [113]:
def export_csv(merge_eyes=False):
  k = range(1, int(524/2), 1)
  cropped_eyes = "drive/MyDrive/datasets/cropped_eyes/"
  datasets_path = "drive/MyDrive/datasets/"
  person_path = []
  data = {
      "image_path": [],
      "class": []
  }
  filename = ""
  if merge_eyes:
    filename = "cropped_eyes_merged.csv"
    pass
  else:
    folders = os.listdir(cropped_eyes)
    filename = "cropped_eyes.csv"
    #folders.sort()
    for folder in folders:
      id = int(folder.replace("C",""))
      pictures = os.listdir(cropped_eyes + folder)
      for picture in pictures:
        picture_path = cropped_eyes + folder + "/" + picture
        data["image_path"].append(picture_path)
        data["class"].append(folder)
      
      #print(id)
  df = pd.DataFrame.from_dict(data)
  print(datasets_path + filename)
  df.to_csv(datasets_path + filename)

  return pd.DataFrame.from_dict(data)

In [ ]:
#@title
target = 3
for comb in ["C1C2","C3C4","C5C1","C7C4"]:
  ids = comb[1:].split("C")
  print(ids)
  if str(target) == ids[0]:
    print([int(ids[0]), int(ids[1])])
    print("achei")
    break


In [ ]:
#@title
def verify_data_integrity(label_path):
  if label_path == "falsely":
    combinations = os.listdir(DATA_CONFIG["falsely_generate"]["dataset_url"])
    issues = []
    print(len(combinations))
    for combination in combinations:
      target = combination[1:].split("C")
      for comb in combinations:
        ids = combination[1:].split("C")
        #print("{} {} {} {}".format(target[0] , ids[0] , target[1],ids[1]))
        if target[0] == ids[1] and target[1] == ids[0]:
          issues.append(combination)
    print(issues)
    issues = []
    
    for combination in combinations:
      files = os.listdir(DATA_CONFIG["falsely_generate"]["dataset_url"] + "{}".format(combination))
      if len(files) < 2:
        print(files)
        issues.append(combination)
    print(issues)

In [ ]:
#@title

img = cv2.imread("drive/MyDrive/ubirisv2/CLASSES_400_300_Part1/C256_S1_I13.tiff")
a = apply_blur_on_image(img)
print_image(a)

In [ ]:
#@title
img = cv2.imread("drive/MyDrive/ubirisv2/CLASSES_400_300_Part1/C256_S1_I13.tiff")
print_image(img)
im = detect_best_circle_in(img, canny_config=(10,20))
print_image(im)

In [ ]:
#@title
create_dataset()

In [ ]:
#@title
img = cv2.imread("drive/MyDrive/ubirisv2/CLASSES_400_300_Part2/C430_S1_I13.tiff")
imgv2_2_work=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
detect_best_circle_in(imgv2_2_work)

In [ ]:
img = cv2.imread("drive/MyDrive/ubirisv2/CLASSES_400_300_Part1/C55_S1_I14.tiff")
img_2_work=cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
canny = canny_edge_detect(img_2_work, 30,34)
print_image(canny)
img_2_work = apply_contourns(img_2_work, canny)
mask = find_best_circle(img_2_work, output_image = img.copy(),print_output=True, show_circles=True)
iris_rounded = apply_mask(mask,img_2_work.copy())
print("target image above")
#print_image(iris_rounded)

In [ ]:
picture = "C55_S1_I14.tiff"
image_path = "drive/MyDrive/ubirisv2/CLASSES_400_300_Part1/" + picture
cut_eyes = "drive/MyDrive/datasets/cut_eyes/"

original = cv2.imread(image_path)
original =cv2.cvtColor(original, cv2.COLOR_RGB2BGR)
iris = detect_best_circle_in(original, canny_config=(30,44))
print_image(iris)
save = input("Save? y/n: ")
if save:
  save_image(cut_eyes + "C{}/".format(55), picture,cv2.cvtColor(iris, cv2.COLOR_BGR2RGB))

In [114]:
export_csv()

drive/MyDrive/datasets/cropped_eyes.csv


,image_path,class
0,drive/MyDrive/datasets/cropped_eyes/C105/C105_...,C105
1,drive/MyDrive/datasets/cropped_eyes/C105/C105_...,C105
2,drive/MyDrive/datasets/cropped_eyes/C105/C105_...,C105
3,drive/MyDrive/datasets/cropped_eyes/C105/C105_...,C105
4,drive/MyDrive/datasets/cropped_eyes/C105/C105_...,C105
...,...,...
1938,drive/MyDrive/datasets/cropped_eyes/C99/C99_S2...,C99
1939,drive/MyDrive/datasets/cropped_eyes/C99/C99_S1...,C99
1940,drive/MyDrive/datasets/cropped_eyes/C99/C99_S1...,C99
1941,drive/MyDrive/datasets/cropped_eyes/C99/C99_S1...,C99
